# Project 3D Modeling and Image Based Rendering

## 0 Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# General imports
import cv2
import numpy as np
from glob import glob
import os.path
from os import makedirs

BASE_DATA_PATH = "/Users/kobe/Documents/School/2024-2025/3D_Modeling_and_Image_Based_Rendering/project/dataset"
CURRENT_DATASET = BASE_DATA_PATH + "/GrayCodes_HighRes"
CHESS_PATH = CURRENT_DATASET + "/chess"
RAW_PATH = CURRENT_DATASET + "/raw"
UNDIST_PATH = CURRENT_DATASET + "/undistorted"
PATTERN_PATH = CURRENT_DATASET + "/patterns"

if not os.path.exists(UNDIST_PATH):
	makedirs(UNDIST_PATH)

if not os.path.exists(PATTERN_PATH):
	makedirs(PATTERN_PATH)

In [3]:
def getFilesSortedNumeric(glob_pattern: str):
	files = glob(glob_pattern)
	return sorted(files, key=lambda f: int(os.path.splitext(os.path.basename(f))[0]))

## 1 Camerakalibratie

### Kalibreer camera

In [ ]:
from calibration import combine_extrinsic_vecs, intrinsic_calibration

chess_files = glob(f"{CHESS_PATH}/*.jpg")
error, intrinsic, distortion, rotation_matrices, translation_vectors, image_size = intrinsic_calibration(chess_files, (7,9))
extrinsics = combine_extrinsic_vecs(rotation_matrices, translation_vectors)

np.savez(f"{CURRENT_DATASET}/camCalibration.npz", intrinsic=intrinsic, distortion=distortion, extrinsics=extrinsics, img_size=image_size)

### Load kalibration data

In [5]:
calibration = np.load(f"{CURRENT_DATASET}/camCalibration.npz")
intrinsic = calibration["intrinsic"]
distortion = calibration["distortion"]
extrinsics = calibration["extrinsics"]
width, height = calibration['img_size']

### Undistort alle afbeelding

In [3]:
from undistort import undistortAllViews

undistortAllViews(RAW_PATH, UNDIST_PATH, intrinsic, distortion)

Undistorting view1
Undistorting 40.jpg
Undistorting view0
Undistorting 40.jpg


### Visualize poses

In [ ]:
from visualizeCameraPoints import drawCameraPoints

drawCameraPoints(height, width, intrinsic, extrinsics)

## 2 Structured Light

### Genereer patronen

In [ ]:
from GrayCodeEncoder import GrayCodeEncoder


ROWS = 1080
COLS = 1920
DEPTH = 10

encoder = GrayCodeEncoder(ROWS, COLS, DEPTH)
for i, pattern in enumerate(encoder.patterns):
	output_filename = f"{PATTERN_PATH}/{i}.jpg"
	cv2.imwrite(output_filename, pattern)
	print(f"\rSaved {os.path.basename(output_filename)}", end='')


### Decode graycode images

In [ ]:
from GrayCodeDecoder import GrayCodeDecoder

THRESHOLD = 8
for view in os.listdir(str(UNDIST_PATH)):
	print(f"Decoding {view}...")
	decoder = GrayCodeDecoder([cv2.imread(file, cv2.IMREAD_GRAYSCALE) for file in getFilesSortedNumeric(f"{UNDIST_PATH}/{view}/[0-9][0-9].jpg")])
	codes, mask = decoder.decode(THRESHOLD)
	np.savez_compressed(f"{CURRENT_DATASET}/{view}_decoded.npz", codes=codes, mask=mask, threshold=THRESHOLD)

Decoding view1...
Decoding 20/20
(3168, 4752) (3168, 4752)
Decoding view0...
Decoding 20/20
(3168, 4752) (3168, 4752)


### Load decoded data

In [5]:
def createViewDict(view: str):
	data = np.load(f"{CURRENT_DATASET}/{view}_decoded.npz")
	return {
		"codes": data["codes"],
		"mask": data["mask"],
		"threshold": data['threshold']
	}

decoded_views = {
	 f"{view}": createViewDict(view) for view in os.listdir(UNDIST_PATH)
}

### Find matches between views

In [ ]:
from matcher import correspond

kp0, kp1, dMatches, matches = correspond(decoded_views)
np.savez_compressed(f"{CURRENT_DATASET}/matches.npz", matches=matches, allow_pickle=True)

### Load matches

In [8]:
loaded_data = np.load(f"{CURRENT_DATASET}/matches.npz", allow_pickle=True)
matches = loaded_data['matches'].item()
loaded_data.close()

### Draw Matches

In [ ]:
from matcher import drawMatches

img0 = cv2.imread(f"{UNDIST_PATH}/view0/00.jpg")
img1 = cv2.imread(f"{UNDIST_PATH}/view1/00.jpg")
drawMatches(matches, img0, img1)

: 

## 3 Reconstructie Puntenwolk

### Essentiele matrix berekenen

In [15]:
def getKeyPoints(matches):
	kp0 = []
	kp1 = []
	for match in matches.values():
		kp0.append(match[0])
		kp1.append(match[1])
	return np.array(kp0), np.array(kp1)

In [ ]:
from essentialMatrixGeneration import generateEssentialMatrix

kp0, kp1 = getKeyPoints(matches)
essential_matrix, mask = generateEssentialMatrix(kp0, kp1, intrinsic)

### Recover Pose of cameras

In [17]:
from poseRecovery import recoverPose

kp0, kp1 = getKeyPoints(matches)
rotation_cam1, translation_cam1, mask_pose = recoverPose(essential_matrix, kp0, kp1, intrinsic, mask)

### Visualize poses of the cameras

In [19]:
from calibration import combine_extrinsic_vec
from visualizeCameraPoints import drawCameraPoints


cam0_extrinsic = combine_extrinsic_vec(np.eye(3), np.zeros((3,1)))
cam1_extrinsic = combine_extrinsic_vec(rotation_cam1, translation_cam1)
drawCameraPoints(height, width, intrinsic, [cam0_extrinsic, cam1_extrinsic])

### Triangulate points

In [45]:
from triangulation import triangulatePoints, triangulatePointsCustom

kp0, kp1 = getKeyPoints(matches)
points_4d_hom = triangulatePoints(intrinsic, rotation_cam1, translation_cam1, kp0.T, kp1.T)
proj0 = intrinsic @ np.hstack((np.eye(3), np.zeros((3,1))))
proj1 = intrinsic @ np.hstack((rotation_cam1, translation_cam1))
points_4d_hom_custom = triangulatePointsCustom(proj0, proj1, kp0, kp1)

split projection matrices


### Visualize triangulated points

In [47]:
import open3d
import open3d.visualization
import open3d.cpu.pybind.utility as utility


points3D = points_4d_hom[:3, :] / points_4d_hom[3, :]
points3D_custom = points_4d_hom_custom[:3, :] / points_4d_hom_custom[3, :]

points = utility.Vector3dVector(points3D_custom.T)
point_cloud = open3d.geometry.PointCloud(points)
open3d.visualization.draw_geometries([point_cloud])

## 4 Plane-Sweep

## 5 Eigen Dataset Capteren

## 6 Reconstrucite Camera-Camera-Projector

## 7 Light Field Displays